In [85]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle


# Primeira Execução

In [86]:
rota = 32893
df = pd.read_csv('rota32893.csv')
df.head()

,data_partida,data_chegada,linha,tempo_viagem,equipamento,dia_semana,partidaTimeStamp,qtdDiasAno,direcao,tipo_dia,turno_dia,chegadaTimeStamp,hora,hora_dia
0,2019-07-03 17:42:38,2019-07-03 18:20:59,32893,2301,1351,3,1562186558,184,2,1,3,1562188859,NaN,17
1,2019-07-05 08:53:39,2019-07-05 09:23:33,32893,1794,1353,5,1562327619,186,2,1,2,1562329413,NaN,8
2,2019-07-17 08:08:59,2019-07-17 08:47:22,32893,2303,1353,3,1563361739,198,2,1,2,1563364042,NaN,8
3,2019-07-17 11:43:09,2019-07-17 12:20:55,32893,2266,1353,3,1563374589,198,2,1,2,1563376855,NaN,11
4,2019-07-25 10:01:00,2019-07-25 10:35:59,32893,2099,1353,4,1564059660,206,2,1,2,1564061759,NaN,10


In [87]:
# Dropamos:
# A data_chegada e a chegadaTimeStamp pois com esses valores daria para prever o tempo de viagem(Y)
# linha e equipamento por se tratar apenas de IDs
# hora por conter apenas valores nulos
# direcao por conter apenas o valor 2

df = df.drop(['data_chegada','linha','equipamento','direcao','chegadaTimeStamp','hora'], axis=1)

In [88]:
corr = df.corr()
corr

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\2438084875.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia
tempo_viagem,1.000000,-0.107438,-0.047102,-0.047254,-0.043944,0.011854,0.018935
dia_semana,-0.107438,1.000000,-0.005862,-0.005899,0.059663,-0.000630,0.004323
partidaTimeStamp,-0.047102,-0.005862,1.000000,0.999970,-0.009222,0.041895,0.046805
qtdDiasAno,-0.047254,-0.005899,0.999970,1.000000,-0.009330,0.034703,0.039107
tipo_dia,-0.043944,0.059663,-0.009222,-0.009330,1.000000,0.011343,0.013525
turno_dia,0.011854,-0.000630,0.041895,0.034703,0.011343,1.000000,0.934388
hora_dia,0.018935,0.004323,0.046805,0.039107,0.013525,0.934388,1.000000


In [89]:
df[['hora_dia','turno_dia']].corr()

,hora_dia,turno_dia
hora_dia,1.000000,0.934388
turno_dia,0.934388,1.000000


In [90]:
df[['partidaTimeStamp','qtdDiasAno']].corr()

,partidaTimeStamp,qtdDiasAno
partidaTimeStamp,1.00000,0.99997
qtdDiasAno,0.99997,1.00000


In [91]:
#Vamos tirar hora_dia pois tem uma alta correlação com turno dia, escolhemos essa coluna pois os valores são apenas de 1 a 4 enquanto hora é de 0 a 23
df = df.drop(['qtdDiasAno'], axis=1)

In [92]:
#Ordenando pela Data
df = df.sort_values(by=["data_partida"], ascending=False)

In [93]:
#Separamos treino e teste
shape = int(df.shape[0] * 0.3)
df_teste = df[:shape]
df_treino = df[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (3691, 7)      Teste:(1581, 7)


In [94]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_treino = df_treino[['tempo_viagem']]

In [95]:
params_xgb = {
        "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
        "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
        "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
        "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
        "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
        }

# Hyperparâmetros do Random Forest
params_rf = {
        "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
        "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
        "max_depth": list(range(2, 15)),#The maximum depth of the tree.
        "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
        "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
        "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
        }

In [96]:
# xgb_rand_search.best_estimator_

In [97]:
# rf_rand_search.best_estimator_

In [98]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
tempo_rf = end_rf - start_rf
tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
tempo_lr = end_lr - start_lr
tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

[21:38:16] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\995122566.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [99]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1876206521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [100]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr"])

In [101]:
metrics = []

In [102]:
metrics.append(['Primeira execução','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Primeira execução','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Primeira execução','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics

[['Primeira execução',
  'RRSE',
  0.9774835759541218,
  1.0070681999232278,
  1.0076720195224023],
 ['Primeira execução',
  'MAPE',
  0.24087418998166102,
  0.2608166271498693,
  0.22150027759041008],
 ['Primeira execução', 'Tempo', 1.27131, 1.879889, 0.2999]]

In [103]:
df = df.merge(df_teste[['predicao_xgb', 'predicao_lr', 'predicao_rf']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf
4080,2019-09-30 23:01:36,1702,1,1569895296,1,4,23,1716.796387,2703.159793,1747.722285
5053,2019-09-30 22:40:17,1671,1,1569894017,1,4,22,1822.618652,2700.136649,1846.534433
3908,2019-09-30 22:30:04,1453,1,1569893404,1,4,22,1822.618652,2700.163530,1846.534433
4343,2019-09-30 21:50:20,1867,1,1569891020,1,4,21,1822.618652,2697.188843,1794.354328
4410,2019-09-30 21:38:06,1590,1,1569890286,1,4,21,1822.618652,2697.221031,1794.354328


# Teste com Clima

In [104]:
#Lendo dados de clima e convertendo a data pra datetime (na leitura veio como objeto)
clima = pd.read_csv('clima.csv')
clima['Datetime'] = pd.to_datetime(clima['Datetime'])

In [105]:
#copiando o DAtaframe original para não alterar o primeiro
aux = df.copy()
#Mudanças na data_partida para conseguir fazer o merge
aux["data_partida2"] = pd.to_datetime(aux["data_partida"])
aux["data_partida2"] = aux["data_partida2"].dt.round("H")
#Merge dos dados e exclusão da coluna auxiliar
df_clima = aux.merge(clima, left_on="data_partida2", right_on="Datetime", how="left")
df_clima.drop(['data_partida2','Datetime'],axis=1,inplace=True)
df_clima.index = aux.index
df_clima.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor
4080,2019-09-30 23:01:36,1702,1,1569895296,1,4,23,1716.796387,2703.159793,1747.722285,0,1
5053,2019-09-30 22:40:17,1671,1,1569894017,1,4,22,1822.618652,2700.136649,1846.534433,0,1
3908,2019-09-30 22:30:04,1453,1,1569893404,1,4,22,1822.618652,2700.163530,1846.534433,0,1
4343,2019-09-30 21:50:20,1867,1,1569891020,1,4,21,1822.618652,2697.188843,1794.354328,0,1
4410,2019-09-30 21:38:06,1590,1,1569890286,1,4,21,1822.618652,2697.221031,1794.354328,0,1


In [106]:
#Separamos treino e teste
shape = int(df_clima.shape[0] * 0.3)
df_teste = df_clima[:shape]
df_treino = df_clima[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (3691, 12)      Teste:(1581, 12)


In [107]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_treino = df_treino[['tempo_viagem']]

In [108]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
tempo_rf = end_rf - start_rf
tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
tempo_lr = end_lr - start_lr
tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

[21:38:19] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\995122566.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [109]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb_clima"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr_clima"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf_clima"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb_clima"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr_clima"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_27608\1375954795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [110]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb_clima"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf_clima"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr_clima"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr_clima"])

In [111]:

metrics.append(['Teste Clima','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Teste Clima','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Teste Clima','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics

[['Primeira execução',
  'RRSE',
  0.9774835759541218,
  1.0070681999232278,
  1.0076720195224023],
 ['Primeira execução',
  'MAPE',
  0.24087418998166102,
  0.2608166271498693,
  0.22150027759041008],
 ['Primeira execução', 'Tempo', 1.27131, 1.879889, 0.2999],
 ['Teste Clima',
  'RRSE',
  0.9741492810353677,
  0.978555728681605,
  1.0051216627664927],
 ['Teste Clima',
  'MAPE',
  0.2390908566717386,
  0.24558989509696494,
  0.22533254684809412],
 ['Teste Clima', 'Tempo', 0.924051, 1.64889, 0.2997]]

In [112]:
df = df.merge(df_teste[['Descricao Chuva','Calor','predicao_xgb_clima', 'predicao_lr_clima', 'predicao_rf_clima']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor,predicao_xgb_clima,predicao_lr_clima,predicao_rf_clima
4080,2019-09-30 23:01:36,1702,1,1569895296,1,4,23,1716.796387,2703.159793,1747.722285,0,1,1705.044800,2693.718975,1907.642863
5053,2019-09-30 22:40:17,1671,1,1569894017,1,4,22,1822.618652,2700.136649,1846.534433,0,1,1838.380493,2693.656903,1967.046012
3908,2019-09-30 22:30:04,1453,1,1569893404,1,4,22,1822.618652,2700.163530,1846.534433,0,1,1838.380493,2693.684513,1967.046012
4343,2019-09-30 21:50:20,1867,1,1569891020,1,4,21,1822.618652,2697.188843,1794.354328,0,1,1822.235840,2693.672213,1951.932231
4410,2019-09-30 21:38:06,1590,1,1569890286,1,4,21,1822.618652,2697.221031,1794.354328,0,1,1822.235840,2693.705273,1951.932231


# Tempos Anteriores

In [113]:
df_tempos = df.copy()
df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem"]

df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem_1"].shift(1)
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem_2"].shift(2)
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem_3"].shift(3)
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem_4"].shift(4)
df_tempos.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,Descricao Chuva,Calor,predicao_xgb_clima,predicao_lr_clima,predicao_rf_clima,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4
4080,2019-09-30 23:01:36,1702,1,1569895296,1,4,23,1716.796387,2703.159793,1747.722285,0.0,1.0,1705.044800,2693.718975,1907.642863,NaN,NaN,NaN,NaN
5053,2019-09-30 22:40:17,1671,1,1569894017,1,4,22,1822.618652,2700.136649,1846.534433,0.0,1.0,1838.380493,2693.656903,1967.046012,1702.0,NaN,NaN,NaN
3908,2019-09-30 22:30:04,1453,1,1569893404,1,4,22,1822.618652,2700.163530,1846.534433,0.0,1.0,1838.380493,2693.684513,1967.046012,1671.0,1702.0,NaN,NaN
4343,2019-09-30 21:50:20,1867,1,1569891020,1,4,21,1822.618652,2697.188843,1794.354328,0.0,1.0,1822.235840,2693.672213,1951.932231,1453.0,1671.0,1702.0,NaN
4410,2019-09-30 21:38:06,1590,1,1569890286,1,4,21,1822.618652,2697.221031,1794.354328,0.0,1.0,1822.235840,2693.705273,1951.932231,1867.0,1453.0,1671.0,1702.0


In [114]:
shape = int(df_tempos.shape[0] * 0.3)
df_teste = df_tempos[:shape]
df_treino = df_tempos[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (3691, 19)      Teste:(1581, 19)


In [115]:

for cenario in range(1, 5):
        if cenario == 1:
                df_teste = df_teste.iloc[1:]
                df_treino = df_treino.iloc[1:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 2:
                df_teste = df_teste.iloc[2:]
                df_treino = df_treino.iloc[2:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 3:
                df_teste = df_teste.iloc[3:]
                df_treino = df_treino.iloc[3:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_treino = df_treino["tempo_viagem"]
        else:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_treino = df_treino["tempo_viagem"]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
        tempo_rf = end_rf - start_rf
        tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
        tempo_lr = end_lr - start_lr
        tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")

        # Fizemos as predições e juntamos no DataFrame
        df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf_{cenario}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb_{cenario}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf_{cenario}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr_{cenario}"])

        metrics.append([f'Teste Tempos {cenario}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Tempos {cenario}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Tempos {cenario}','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 

        df = df.merge(df_teste[[f'predicao_xgb_{cenario}', f'predicao_lr_{cenario}', f'predicao_rf_{cenario}']], how="left", left_index=True, right_index=True)

[21:38:22] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:38:25] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:38:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not 

In [116]:
metrics


[['Primeira execução',
  'RRSE',
  0.9774835759541218,
  1.0070681999232278,
  1.0076720195224023],
 ['Primeira execução',
  'MAPE',
  0.24087418998166102,
  0.2608166271498693,
  0.22150027759041008],
 ['Primeira execução', 'Tempo', 1.27131, 1.879889, 0.2999],
 ['Teste Clima',
  'RRSE',
  0.9741492810353677,
  0.978555728681605,
  1.0051216627664927],
 ['Teste Clima',
  'MAPE',
  0.2390908566717386,
  0.24558989509696494,
  0.22533254684809412],
 ['Teste Clima', 'Tempo', 0.924051, 1.64889, 0.2997],
 ['Teste Tempos 1',
  'RRSE',
  0.9691222751753616,
  0.9725227386597868,
  0.9732500666455918],
 ['Teste Tempos 1',
  'MAPE',
  0.2110252677092281,
  0.24554505727335854,
  0.21494024248014565],
 ['Teste Tempos 1', 'Tempo', 0.900998, 2.258954, 0.1982],
 ['Teste Tempos 2',
  'RRSE',
  0.9606897372934411,
  0.9660333571659016,
  0.9654077515508571],
 ['Teste Tempos 2',
  'MAPE',
  0.23231647930998087,
  0.24986653457023525,
  0.21323552828092898],
 ['Teste Tempos 2', 'Tempo', 0.928019, 2.517

In [117]:
df = df.merge(df_teste[['tempo_viagem_1','tempo_viagem_2','tempo_viagem_3','tempo_viagem_4']], how="left", left_index=True, right_index=True)
df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,...,predicao_rf_1,predicao_xgb_2,predicao_lr_2,predicao_rf_2,predicao_xgb_3,predicao_lr_3,predicao_rf_3,predicao_xgb_4,predicao_lr_4,predicao_rf_4
4409,2019-09-30 19:31:11,4629,1,1569882671,1,4,19,2776.276123,2691.396503,2650.674565,...,2862.739060,2778.701660,2475.514450,3060.388835,2855.292480,2422.768731,3028.032163,2786.042969,2386.621498,3056.825899
3904,2019-09-30 19:25:49,4446,1,1569882349,1,4,19,2776.276123,2691.410624,2650.674565,...,2756.219371,2717.272949,2954.136806,3038.161720,2794.687012,2849.812974,3073.783449,2692.066895,2794.743063,3002.163462
4362,2019-09-30 18:59:01,2161,1,1569880741,1,4,18,2971.267334,2688.401907,2756.010041,...,3117.505355,3132.152832,3252.452590,3299.964096,3135.760742,3142.203316,3150.748184,3110.003174,3062.804237,3030.725568
5189,2019-09-30 18:54:06,4261,1,1569880446,1,4,18,2971.267334,2688.414843,2756.010041,...,2894.874661,3006.432129,2844.823104,2772.130468,3164.144043,3027.141672,3017.544043,3016.517578,2952.743256,3061.277380
4255,2019-09-30 18:25:56,2937,1,1569878756,1,4,18,2971.267334,2688.488953,2756.010041,...,3077.875210,3036.472656,2900.007487,3153.916730,3130.833252,3075.132905,3048.505601,3045.065674,3174.424337,3066.624789


# Testes Intervalos de Teste_Treino

In [118]:
#Separamos treino e teste
df_intervalos = df.copy()
df_intervalos["data_partida2"] = pd.to_datetime(df_intervalos.index)

df_teste.head()

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf,...,predicao_rf_1,predicao_xgb_2,predicao_lr_2,predicao_rf_2,predicao_xgb_3,predicao_lr_3,predicao_rf_3,predicao_xgb_4,predicao_lr_4,predicao_rf_4
4409,2019-09-30 19:31:11,4629,1,1569882671,1,4,19,2776.276123,2691.396503,2650.674565,...,2862.739060,2778.701660,2475.514450,3060.388835,2855.292480,2422.768731,3028.032163,2786.042969,2386.621498,3056.825899
3904,2019-09-30 19:25:49,4446,1,1569882349,1,4,19,2776.276123,2691.410624,2650.674565,...,2756.219371,2717.272949,2954.136806,3038.161720,2794.687012,2849.812974,3073.783449,2692.066895,2794.743063,3002.163462
4362,2019-09-30 18:59:01,2161,1,1569880741,1,4,18,2971.267334,2688.401907,2756.010041,...,3117.505355,3132.152832,3252.452590,3299.964096,3135.760742,3142.203316,3150.748184,3110.003174,3062.804237,3030.725568
5189,2019-09-30 18:54:06,4261,1,1569880446,1,4,18,2971.267334,2688.414843,2756.010041,...,2894.874661,3006.432129,2844.823104,2772.130468,3164.144043,3027.141672,3017.544043,3016.517578,2952.743256,3061.277380
4255,2019-09-30 18:25:56,2937,1,1569878756,1,4,18,2971.267334,2688.488953,2756.010041,...,3077.875210,3036.472656,2900.007487,3153.916730,3130.833252,3075.132905,3048.505601,3045.065674,3174.424337,3066.624789


In [119]:
data_atual = df_intervalos["data_partida2"].dt.date.min() + timedelta(days=14)
dias_treino = 14
dias_teste = 79


while data_atual < (df_intervalos["data_partida2"].dt.date.max() - timedelta(days=14)):
        df_teste = df_intervalos[df_intervalos["data_partida2"].dt.date > data_atual]
        df_treino = df_intervalos[df_intervalos["data_partida2"].dt.date <= data_atual]

        #Separamos a classe dos atributos 
        x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_teste = df_teste[['tempo_viagem']]
        x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_treino = df_treino[['tempo_viagem']]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_xgb = float(f"{str(tempo_xgb.seconds)}.{str(tempo_xgb.microseconds)}")
        tempo_rf = end_rf - start_rf
        tempo_rf = float(f"{str(tempo_rf.seconds)}.{str(tempo_rf.microseconds)}")
        tempo_lr = end_lr - start_lr
        tempo_lr = float(f"{str(tempo_lr.seconds)}.{str(tempo_lr.microseconds)}")
        
        # Fizemos as predições e juntamos no DataFrame 
        df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr {dias_treino} {dias_teste}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf {dias_treino} {dias_teste}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb {dias_treino} {dias_teste}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf {dias_treino} {dias_teste}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr {dias_treino} {dias_teste}"])

        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','Tempo', tempo_xgb, tempo_rf, tempo_lr])

        df = df.merge(df_teste[[f'predicao_xgb {dias_treino} {dias_teste}',f'predicao_rf {dias_treino} {dias_teste}',f'predicao_lr {dias_treino} {dias_teste}']], how="left", left_index=True, right_index=True)
        

        dias_teste -= 14
        dias_treino += 14

        data_atual += timedelta(days=14)



In [120]:
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv(f'metricas{rota}.csv',index=False)
metrics

,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Primeira execução,RRSE,0.977484,1.007068,1.007672
1,Primeira execução,MAPE,0.240874,0.260817,0.221500
2,Primeira execução,Tempo,1.271310,1.879889,0.299900
3,Teste Clima,RRSE,0.974149,0.978556,1.005122
4,Teste Clima,MAPE,0.239091,0.245590,0.225333
5,Teste Clima,Tempo,0.924051,1.648890,0.299700
6,Teste Tempos 1,RRSE,0.969122,0.972523,0.973250
7,Teste Tempos 1,MAPE,0.211025,0.245545,0.214940
8,Teste Tempos 1,Tempo,0.900998,2.258954,0.198200
9,Teste Tempos 2,RRSE,0.960690,0.966033,0.965408


In [121]:
df.to_csv(f'previsão_{rota}.csv',index=False)

In [122]:
df.shape

(5272, 31)